<a href="https://colab.research.google.com/github/JoergNeumann/GenAI/blob/main/Code_Interpreter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Interpreter
Beispiel für die Verwendung der OpenAI Assistant API und des Code Interpreter-Features.

## Setup

In [ ]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

## Assistant erstellen

In [1]:
import openai
import os
import time

# OpenAI API Key aus Colab Secret auslesen und OpenAI Client erstellen
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=api_key)

assistant = client.beta.assistants.create(
  name="Technical Assistant",
  instructions="You helps people with technical issues.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4o",
)

## Thread erstellen

In [2]:
thread = client.beta.threads.create()

## Anweisung absetzen

In [3]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="create a qr-code for 'www.neogeeks.de' and give it back to me."
)

## Run und Event Handler erstellen

In [4]:
from typing_extensions import override
from openai import AssistantEventHandler

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)


with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()



assistant > code_interpreter

import qrcode

# Create a QR code for the given URL
input_data = "www.neogeeks.de"
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
qr.add_data(input_data)
qr.make(fit=True)

# Create an image from the QR Code instance
img = qr.make_image(fill_color="black", back_color="white")

# Save the image
img_path = '/mnt/data/neogeeks_qr.png'
img.save(img_path)

img_path
assistant > I've created the QR code for 'www.neogeeks.de'. You can download it using the link below:

[Download the QR Code](sandbox:/mnt/data/neogeeks_qr.png)None

##Mit Dateien arbeiten

In [5]:
# Datei hochladen
file_path = "/content/Umsatz.csv"  # Pfad zur hochzuladenden Datei
with open(file_path, "rb") as file:
    uploaded_file = client.files.create(file=file, purpose="assistants")

# Assistenten erstellen
assistant = client.beta.assistants.create(
    name="Data Analyst",
    instructions="Du bist ein Assistent, der Daten analysiert und bewertet.",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
      "code_interpreter": {
        "file_ids": [uploaded_file.id]
      }
    },
    model="gpt-4-1106-preview",
)

# Neuen Thread erstellen
thread = client.beta.threads.create()

# Benutzeranfrage mit Datei-ID hinzufügen
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Bitte ermittle den durchschnittlichen Umsatz in der hochgeladenen Datei.",
)

# Ausführung starten
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

# Auf Abschluss der Ausführung warten
while run.status in ["queued", "in_progress"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# Ergebnisse abrufen
if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for msg in messages.data:
        if msg.role == "assistant":
            for content in msg.content:
                if content.type == "text":
                    print(content.text.value)
                elif content.type == "code":
                    print("Generierter Code:\n", content.code.value)
                elif content.type == "image_file":
                    print("Generiertes Bild:", content.image_file.file_id)
else:
    print("Die Ausführung ist fehlgeschlagen.")


Der durchschnittliche Umsatz in der hochgeladenen Datei beträgt etwa 276.428,57 €.
Die Datei scheint ein semikolon-getrenntes CSV zu sein. Ich werde die Daten mit der korrekten Trennung einlesen, und dann werde ich den durchschnittlichen Umsatz berechnen. Dabei ist zu beachten, dass die Umsatzwerte in der Spalte "Umsatz" als Text und mit einem Euro-Zeichen am Ende vorliegen. Ich muss also diese Zeichen entfernen und die Zahlen in einen numerischen Typ umwandeln, bevor ich den Durchschnitt berechnen kann. Ich werde nun mit diesem Prozess fortfahren.
Zuerst muss ich herausfinden, in welchem Format die Datei vorliegt. Basierend darauf kann ich entscheiden, wie ich die Daten einlese und den Durchschnittsberechnung durchführe. Ich werde mit der Untersuchung der Datei beginnen, um festzustellen, welches Format sie hat und welche Daten sie enthält.




## OCR anwenden

In [7]:
# Datei hochladen
file_path = "/content/R240004.png"  # Pfad zur hochzuladenden Datei
with open(file_path, "rb") as file:
    invoice_file = client.files.create(file=file, purpose="assistants")

# Assistenten erstellen
assistant = client.beta.assistants.create(
    name="Data Analyst",
    instructions="Du bist ein Assistent, der Daten analysiert und bewertet.",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
      "code_interpreter": {
        "file_ids": [invoice_file.id]
      }
    },
    model="gpt-4-1106-preview",
)

# Neuen Thread erstellen
thread = client.beta.threads.create()

# Benutzeranfrage mit Datei-ID hinzufügen
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Bitte ermittle die Rechnungsnummer in der hochgeladenen Datei.",
)

# Ausführung starten
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

# Auf Abschluss der Ausführung warten
while run.status in ["queued", "in_progress"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# Ergebnisse abrufen
if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for msg in messages.data:
        if msg.role == "assistant":
            for content in msg.content:
                if content.type == "text":
                    print(content.text.value)
                elif content.type == "code":
                    print("Generierter Code:\n", content.code.value)
                elif content.type == "image_file":
                    print("Generiertes Bild:", content.image_file.file_id)
else:
    print("Die Ausführung ist fehlgeschlagen.")

Die OCR-Texterkennung hat erfolgreich Text aus dem Bild extrahiert. Hier ist ein Teil des Textes, der die Rechnungsnummer enthält:

```
N NEOGEEKS

Testmann GmbH Rechnung R240004
...
```

Die Rechnungsnummer lautet: R240004

Bitte lassen Sie mich wissen, ob Sie weitere Informationen oder Aktionen in Bezug auf diese Datei benötigen.
Die ersten Bytes der Datei deuten darauf hin, dass es sich um eine PNG-Bilddatei handelt, da sie mit der typischen PNG-Signatur "\x89PNG\r\n\x1a\n" beginnt. Das bedeutet, dass ich das Bild laden und möglicherweise Texterkennung (Optical Character Recognition, OCR) verwenden muss, um die Rechnungsnummer aus dem Bild zu extrahieren.

Ich werde nun das Bild mit geeigneten OCR-Tools verarbeiten, um nach der Rechnungsnummer zu suchen. Dafür werde ich die OCR-Engine Tesseract über die Pytesseract-Bibliothek verwenden. Lassen Sie mich diesen Schritt durchführen.
Die hochgeladene Datei scheint keine Erweiterung zu haben, was es schwierig macht, den Dateityp direkt z